In [30]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input,MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [31]:
tf.__version__
tf.config.list_physical_devices('GPU')

[]

In [32]:
(x_train, y_train), (x_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [33]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
x_test, x_train = x_test / 255.0, x_train / 255.0
x_train.shape


(50000, 32, 32, 3)

In [34]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', strides=2, kernel_regularizer = l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', strides=1, kernel_regularizer = l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', strides=1, kernel_regularizer = l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', strides=1, kernel_regularizer = l2(0.01)),
    MaxPooling2D(pool_size=(2, 2)),

    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(class_names), activation='softmax')
])
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
from tensorflow.keras.callbacks import ReduceLROnPlateau

optimizer = Adam(lr = 0.001)
# Define the callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',        # Monitor validation loss
    factor=0.1,                # Reduce learning rate by a factor of 0.1
    patience=10,               # Wait for 10 epochs before reducing learning rate
    min_lr=1e-6,               # Minimum learning rate
    verbose=1                  # Print messages about learning rate reductions
)

In [ ]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
datagen.fit(x_train)

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size= 128), epochs=500, steps_per_epoch=x_train.shape[0]//128,validation_data=(x_valid, y_valid), callbacks = [reduce_lr])

Epoch 1/500


<ipython-input-35-ac2912f0e678>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(x_train, y_train, batch_size= 128), epochs=500, steps_per_epoch=x_train.shape[0]//128,validation_data=(x_valid, y_valid), callbacks = [reduce_lr])


312/312 [==============================] - 47s 148ms/step - loss: 2.4529 - accuracy: 0.2283 - val_loss: 2.0489 - val_accuracy: 0.0510 - lr: 0.0010
Epoch 2/500
312/312 [==============================] - 46s 146ms/step - loss: 2.0728 - accuracy: 0.1062 - val_loss: 2.0006 - val_accuracy: 0.0533 - lr: 0.0010
Epoch 3/500
312/312 [==============================] - 45s 145ms/step - loss: 2.0322 - accuracy: 0.0947 - val_loss: 1.9697 - val_accuracy: 0.0229 - lr: 0.0010
Epoch 4/500
312/312 [==============================] - 45s 146ms/step - loss: 1.9977 - accuracy: 0.0991 - val_loss: 1.9025 - val_accuracy: 0.1706 - lr: 0.0010
Epoch 5/500
312/312 [==============================] - 46s 146ms/step - loss: 1.9663 - accuracy: 0.0970 - val_loss: 1.8726 - val_accuracy: 0.0339 - lr: 0.0010
Epoch 6/500
312/312 [==============================] - 45s 145ms/step - loss: 1.9511 - accuracy: 0.0952 - val_loss: 1.8303 - val_accuracy: 0.1084 - lr: 0.0010
Epoch 7/500
312/312 [==============================] - 45s

In [ ]:
# history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), batch_size=1000)

In [ ]:
import random
sum = 0
for i in range(10):
  ch = random.choice(range(x_test.shape[0]))
  sum += model.predict(x_test[ch].reshape(1,32,32,3)).argmax() == y_test[ch]
  print(f"Prediction  {class_names[model.predict(x_test[ch].reshape(1,32,32,3)).argmax()]} for {class_names[y_test[ch][0]]} ")

print(sum)

In [ ]:
y_test[0]

In [ ]:
import pandas as pd
pd.DataFrame(history.history).plot(figsize=(8, 5))